In [1]:
from DbManager import DbManager
from ApiClient import ApiClient
import numpy as np
import pandas as pd
from  datetime import date

In [3]:
db = DbManager("root", "bandtec123", "localhost", "processamento_db")
api = ApiClient("http://localhost")

data_frame = db.read("SELECT * FROM eventos").astype({
        'categoria':'category',
        'sexo' : 'category',
        'dataNascimento':'datetime64[ns]'
        })
data_frame['categoria'] = data_frame['categoria'].cat.codes
data_frame['sexo'] = data_frame['sexo'].cat.codes
data_frame['idade'] = date.today().year - data_frame['dataNascimento'].dt.year - (
data_frame['dataNascimento'].apply( lambda x : (date.today().month, date.today().day) < (x.month, x.day)) )
quartis_idade = data_frame['idade'].quantile([0.25, 0.5, 0.75])
data_frame['faixa_etaria'] = np.where(
        data_frame['idade'].between(0, quartis_idade[0.25]), 'Primeira Faixa',
        np.where(data_frame['idade'].between(quartis_idade[0.25], quartis_idade[0.5]), 
        'Segunda Faixa',
        'Terceira Faixa')
 )

max_categoria_faixa = data_frame[data_frame['statusEvento'] == 1].groupby(['faixa_etaria', 'categoria']).agg(
        maxPreco = ('preco' , 'max'),
        Quantidade = ('idEvento' , 'count')
        ).reset_index()

Conectado ao banco


In [4]:
max_categoria_faixa

,faixa_etaria,categoria,maxPreco,Quantidade
0,Primeira Faixa,0,2579.35,160
1,Primeira Faixa,1,2757.93,160
2,Primeira Faixa,2,3029.21,174
3,Primeira Faixa,3,2755.14,162
4,Primeira Faixa,4,2832.96,145
5,Primeira Faixa,5,2538.59,137
6,Segunda Faixa,0,2712.74,141
7,Segunda Faixa,1,2682.73,137
8,Segunda Faixa,2,2726.17,143
9,Segunda Faixa,3,2765.59,149
